In [1]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

# Creating list to append tweet data to
tweets_list2 = []
term = "Game of Thrones"
since = "2020-06-01"
until = "2020-06-02"
country = "USA"
# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('"'+term+'" since:'+
                                                        since+" until:"+until+ " near:"+country).get_items()):
    continue
#     if i>1500:
#         break
#     tweets_list2.append([tweet.date, tweet.id, tweet.content, tweet.username])
num = i
# Creating a dataframe from the tweets list above
# tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])
# tweets_df2.to_csv("sample.csv")
num

23

In [20]:
df = pd.read_excel("input-refined.xlsx")
df.head()

,name,language,plot,story,actor1,actor2,actor3,director1,director2,director3,...,show_writer,wiki_url,based_on_book_by,release_date,seven_days_later,num_India,num_USA,num_Germany,num_Spain,num_Mexico
0,13 Reasons Why,English,"Follows teenager Clay Jensen, in his quest to ...","Thirteen Reasons Why, based on the best-sellin...",Dylan Minnette,Christian Navarro,Alisha Boe,Jessica Yu,Kyle Patrick Alvarez,Gregg Araki,...,Brian Yorkey,https://en.wikipedia.org/wiki/13_Reasons_Why,Jay Asher,2017-03-31,2017-04-07,NaN,NaN,NaN,NaN,NaN
1,21 Sarfarosh Saragarhi 1897,Hindi\n|\nEnglish,A chronicle of the lives and events that led t...,A chronicle of the lives and events that led t...,Harry Chauhan,Paras Raajj Ganndhhi,Mohit Raina,Prashant Singh,Raj Acharya,NaN,...,Abhimanyu Raj Singh,https://en.wikipedia.org/wiki/21_Sarfarosh_-_S...,NaN,2018-02-12,2018-02-19,NaN,NaN,NaN,NaN,NaN
2,3Below Tales of Arcadia,English,A pair of teenage royals and their bodyguard e...,A pair of teenage royals and their bodyguard e...,Tatiana Maslany,Diego Luna,Nick Offerman,Johane Matte,Andrew L. Schmidt,Elaine Bogan,...,Guillermo del Toro,https://en.wikipedia.org/wiki/3Below:_Tales_of...,Guillermo del Toro and Daniel Kraus,2018-12-21,2018-12-28,NaN,NaN,NaN,NaN,NaN
3,7Seeds,NaN,"In the immediate future, a giant meteorite has...","In the immediate future, a giant meteorite has...",Scott Gibbs,Alejandro Saab,Jad Saxton,Yukio Takahashi,NaN,NaN,...,NaN,https://en.wikipedia.org/wiki/7_Seeds,NaN,2019-06-28,2019-07-05,NaN,NaN,NaN,NaN,NaN
4,A Series of Unfortunate Events,English,After the loss of their parents in a mysteriou...,After the Baudelaire parents die in a terrible...,Neil Patrick Harris,Patrick Warburton,Malina Weissman,Barry Sonnenfeld,Bo Welch,Mark Palansky,...,"Robert Gordon (screenplay), Daniel Handler (bo...",https://en.wikipedia.org/wiki/A_Series_of_Unfo...,Lemony Snicket,2020-09-11,2020-09-18,NaN,NaN,NaN,NaN,NaN


In [5]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta

In [6]:
def get_release_date(url, country="India"):
    res = requests.get(url)
    html_page = res.content
    soup = BeautifulSoup(html_page, "html.parser")
    content = soup.find_all("tr", {"class": "ipl-zebra-list__item release-date-item"})

    for line in content:
        if country in line.text:
            lines = (str(line.text).strip()).split("\n")
            lines = [line[0:18].strip() for line in lines if len(line[0:18].strip())>8 and line != country]
            if len(lines) > 0 and len(lines[0].split()) == 3:
                release_date = datetime.strptime(lines[0], "%d %B %Y")
                week_after_release = release_date + timedelta(days=7)
                return [str(datetime.date(release_date)), str(datetime.date(week_after_release))]
    return ["N/A", "N/A"]

In [14]:
for indx in df["imdb_id"].index:
    url = "https://www.imdb.com/title/" + df["imdb_id"][indx] + "/releaseinfo"
    dates = get_release_date(url)
    df["release_date"][indx] = dates[0]
    df["seven_days_later"][indx] = dates[1]

<ipython-input-14-1bb7ced1ebaa>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["release_date"][indx] = dates[0]
C:\Users\apoor\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-14-1bb7ced1ebaa>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["seven_days_later"][indx] = dates[1]


In [15]:
df.to_excel("input-refined.xlsx")

In [49]:
def get_number_of_tweets(term, since, until, country):
    # tweets_list2 = []
    num = 0
    count = 0
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(term+" since:"+
                                                        since+" until:"+until+" near:"+country).get_items()):
        count=count+1
    return count
    # Using TwitterSearchScraper to scrape data and append tweets to list
#     for i,tweet in enumerate(sntwitter.TwitterSearchScraper('"'+term+'" since:'+
#                                                             since+" until:"+until+ " near:"+country).get_items()):
#         continue
    #     if i>1500:
    #         break
    #     tweets_list2.append([tweet.date, tweet.id, tweet.content, tweet.username])
#     num = i
    # Creating a dataframe from the tweets list above
    # tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])
    # tweets_df2.to_csv("sample.csv")
#     return num

In [69]:
df = pd.read_csv("output1.csv")
df.tail(10)

,Unnamed: 0,name,language,plot,story,actor1,actor2,actor3,director1,director2,...,show_writer,wiki_url,based_on_book_by,release_date,seven_days_later,num_India,num_USA,num_Germany,num_Spain,num_Mexico
413,413,White Lines,English\n|\nSpanish,A woman tries to solve the mysterious death of...,A woman tries to solve the mysterious death of...,Laura Haddock,Nuno Lopes,Marta Milans,Nick Hamm,Luis Prieto,...,Ãlex Pina,https://en.wikipedia.org/wiki/White_Lines_(TV_...,NaN,2020-05-15,2020-05-22,18,137,492,0,106
414,414,Wizards,English,The third installment of DreamWork's 'Tales of...,The third installment of DreamWork's 'Tales of...,Colin O'Donoghue,David Bradley,James Faulkner,Francisco Ruiz-Velasco,Andrew L. Schmidt,...,Ralph Bakshi,https://en.wikipedia.org/wiki/Wizards_and_Warr...,NaN,2016-01-04,2016-01-11,0,0,0,0,0
415,415,Word Party,English,Four adorable animal babies need your help to ...,A vocabulary-building show from The Jim Henson...,Victor Yerrid,Dorien Davies,Misty Rosas,Bret Nelson,Michael Shawn Lewis,...,Alex Rockwell,https://en.wikipedia.org/wiki/Word_Party,NaN,2016-01-04,2016-01-11,0,0,0,0,0
416,416,World of Winx,English,The Winx travel all over the world searching f...,The Winx travel all over the world searching f...,Rebecca Soler,Alysha Deslorieux,Haven Paschall,Iginio Straffi,NaN,...,Iginio Straffi,https://en.wikipedia.org/wiki/World_of_Winx,NaN,2016-01-04,2016-01-11,0,0,0,0,0
417,417,Wu Assassins,English\n|\nCantonese,A warrior chosen as the latest and last Wu Ass...,A warrior chosen as the latest and last Wu Ass...,Iko Uwais,Byron Mann,Li Jun Li,Toa Fraser,Stephen Fung,...,"Tony Krantz, John Wirth",https://en.wikipedia.org/wiki/Wu_Assassins,NaN,2019-08-08,2019-08-15,27,259,68,0,215
418,418,Yeh Ballet,Hindi,"Discovered by an eccentric ballet master, two ...",Yeh Ballet is a Netflix Original Film directed...,Julian Sands,Jim Sarbh,Sarah-Jane Dias,Sooni Taraporevala,NaN,...,NaN,https://en.wikipedia.org/wiki/Yeh_Ballet,NaN,2020-02-24,2020-03-02,22,0,3,0,0
419,419,YooHoo to the Rescue,NaN,"In a series of magical missions, quick-witted ...","In a series of magical missions, quick-witted ...",Ryan Bartley,Kira Buckland,Lucien Dodge,Horak Kim,NaN,...,NaN,https://en.wikipedia.org/wiki/YooHoo_to_the_Re...,Aurora World,2016-01-04,2016-01-11,0,0,0,0,0
420,420,You Are Wanted,German\n|\nEnglish\n|\nSpanish\n|\nFrench\n|\n...,Lukas Franke (Matthias Schweighöfer) is the vi...,Lukas Franke (Matthias Schweighöfer) is the vi...,Matthias Schweighöfer,Alexandra Maria Lara,Catrin Striebeck,Bernhard Jasper,Matthias Schweighöfer,...,"Hanno Hackfort, Bob Konrad, Richard Kropf",https://en.wikipedia.org/wiki/You_Are_Wanted,NaN,2016-01-04,2016-01-11,0,0,0,0,0
421,421,Young Wallander,English,Follow recently graduated police officer Kurt ...,Follow recently graduated police officer Kurt ...,Adam Palsson,Leanne Best,Richard Dillane,Ole Endresen,Jens Jonsson,...,Ben Harris,https://en.wikipedia.org/wiki/Young_Wallander,NaN,2020-09-03,2020-09-10,5,14,35,0,13
422,422,Z The Beginning of Everything,English,"The story of Zelda Sayre Fitzgerald, a brillia...",Z is a biography series based on the life of Z...,Christina Ricci,David Hoflin,Kristine Nielsen,Mike Barker,Neasa Hardiman,...,"Dawn Prestwich, Nicole Yorkin",https://en.wikipedia.org/wiki/Z:_The_Beginning...,NaN,2016-01-04,2016-01-11,0,0,0,0,0


In [90]:
import math
countries = ["India", "USA", "Germany", "Spain", "Mexico"]
for indx in df["name"][343:].index:
    term = df["name"][indx]
    since = str(df["release_date"][indx])
    until = str(df["seven_days_later"][indx])
    for country in countries:
        if (df["num_"+country][indx]) ==0:
            df["num_"+country][indx] = get_number_of_tweets(term, since, until, country)

<ipython-input-90-0d0bf9e55f04>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["num_"+country][indx] = get_number_of_tweets(term, since, until, country)


In [94]:
indx, df["name"][indx], df["release_date"][indx], df["seven_days_later"][indx]


(147, 'Un extraño enemigo', '2016-01-04', '2016-01-11')

In [55]:
df2 = df

In [48]:
df.head()

,name,language,plot,story,actor1,actor2,actor3,director1,director2,director3,...,show_writer,wiki_url,based_on_book_by,release_date,seven_days_later,num_India,num_USA,num_Germany,num_Spain,num_Mexico
0,13 Reasons Why,English,"Follows teenager Clay Jensen, in his quest to ...","Thirteen Reasons Why, based on the best-sellin...",Dylan Minnette,Christian Navarro,Alisha Boe,Jessica Yu,Kyle Patrick Alvarez,Gregg Araki,...,Brian Yorkey,https://en.wikipedia.org/wiki/13_Reasons_Why,Jay Asher,2017-03-31,2017-04-07,4.0,51.0,31.0,0.0,49.0
1,21 Sarfarosh Saragarhi 1897,Hindi\n|\nEnglish,A chronicle of the lives and events that led t...,A chronicle of the lives and events that led t...,Harry Chauhan,Paras Raajj Ganndhhi,Mohit Raina,Prashant Singh,Raj Acharya,NaN,...,Abhimanyu Raj Singh,https://en.wikipedia.org/wiki/21_Sarfarosh_-_S...,NaN,2018-02-12,2018-02-19,0.0,0.0,0.0,0.0,0.0
2,3Below Tales of Arcadia,English,A pair of teenage royals and their bodyguard e...,A pair of teenage royals and their bodyguard e...,Tatiana Maslany,Diego Luna,Nick Offerman,Johane Matte,Andrew L. Schmidt,Elaine Bogan,...,Guillermo del Toro,https://en.wikipedia.org/wiki/3Below:_Tales_of...,Guillermo del Toro and Daniel Kraus,2018-12-21,2018-12-28,0.0,0.0,0.0,0.0,0.0
3,7Seeds,NaN,"In the immediate future, a giant meteorite has...","In the immediate future, a giant meteorite has...",Scott Gibbs,Alejandro Saab,Jad Saxton,Yukio Takahashi,NaN,NaN,...,NaN,https://en.wikipedia.org/wiki/7_Seeds,NaN,2019-06-28,2019-07-05,2.0,9.0,5.0,0.0,7.0
4,A Series of Unfortunate Events,English,After the loss of their parents in a mysteriou...,After the Baudelaire parents die in a terrible...,Neil Patrick Harris,Patrick Warburton,Malina Weissman,Barry Sonnenfeld,Bo Welch,Mark Palansky,...,"Robert Gordon (screenplay), Daniel Handler (bo...",https://en.wikipedia.org/wiki/A_Series_of_Unfo...,Lemony Snicket,2020-09-11,2020-09-18,0.0,18.0,1.0,0.0,13.0


In [16]:
term = "The Rejuviafication"
since = "2020-09-11"
until = "2020-09-18"
country = "USA"
# tweets = (sntwitter.TwitterSearchScraper('"'+term+'" since:'+since+" until:"+until+ " near:"+country).get_items())
count = 0
# for i,tweet in enumerate(sntwitter.TwitterSearchScraper('"'+term+'" since:'+
#                                                         since+" until:"+until+ " near:"+country).get_items()):
for i,tweet in enumerate(sntwitter.TwitterSearchScraper(term+" near:"+country).get_items()):
    count=count+1
count

0

In [15]:
count

220124

In [13]:
df.to_excel("output-main2.xlsx")
# df2.to_excel("output2.xlsx")

In [2]:
df = pd.read_excel("output-main.xlsx", sheet_name=0)
df.head()

,name,language,plot,story,actor1,actor2,actor3,director1,director2,director3,...,show_writer,wiki_url,based_on_book_by,release_date,seven_days_later,num_India,num_USA,num_Germany,num_Spain,num_Mexico
0,13 Reasons Why,English,"Follows teenager Clay Jensen, in his quest to ...","Thirteen Reasons Why, based on the best-sellin...",Dylan Minnette,Christian Navarro,Alisha Boe,Jessica Yu,Kyle Patrick Alvarez,Gregg Araki,...,Brian Yorkey,https://en.wikipedia.org/wiki/13_Reasons_Why,Jay Asher,2017-03-31,2017-04-07,4,51,31,0,48
1,21 Sarfarosh Saragarhi 1897,Hindi\n|\nEnglish,A chronicle of the lives and events that led t...,A chronicle of the lives and events that led t...,Harry Chauhan,Paras Raajj Ganndhhi,Mohit Raina,Prashant Singh,Raj Acharya,NaN,...,Abhimanyu Raj Singh,https://en.wikipedia.org/wiki/21_Sarfarosh_-_S...,NaN,2018-02-12,2018-02-19,0,0,0,0,0
2,3Below Tales of Arcadia,English,A pair of teenage royals and their bodyguard e...,A pair of teenage royals and their bodyguard e...,Tatiana Maslany,Diego Luna,Nick Offerman,Johane Matte,Andrew L. Schmidt,Elaine Bogan,...,Guillermo del Toro,https://en.wikipedia.org/wiki/3Below:_Tales_of...,Guillermo del Toro and Daniel Kraus,2018-12-21,2018-12-28,0,0,0,0,0
3,7Seeds,NaN,"In the immediate future, a giant meteorite has...","In the immediate future, a giant meteorite has...",Scott Gibbs,Alejandro Saab,Jad Saxton,Yukio Takahashi,NaN,NaN,...,NaN,https://en.wikipedia.org/wiki/7_Seeds,NaN,2019-06-28,2019-07-05,2,9,5,0,7
4,A Series of Unfortunate Events,English,After the loss of their parents in a mysteriou...,After the Baudelaire parents die in a terrible...,Neil Patrick Harris,Patrick Warburton,Malina Weissman,Barry Sonnenfeld,Bo Welch,Mark Palansky,...,"Robert Gordon (screenplay), Daniel Handler (bo...",https://en.wikipedia.org/wiki/A_Series_of_Unfo...,Lemony Snicket,2020-09-11,2020-09-18,0,18,1,0,13


In [104]:
countries = ["India", "USA", "Germany", "Spain", "Mexico"]
for indx in df["name"][147:].index:
    term = df["name"][indx]
    since = str(df["release_date"][indx])
    until = str(df["seven_days_later"][indx])
    for country in countries:
        if (df["num_"+country][indx]) ==0:
            df["num_"+country][indx] = get_number_of_tweets(term, since, until, country)

<ipython-input-104-4b3d7fa7138e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["num_"+country][indx] = get_number_of_tweets(term, since, until, country)


In [6]:
import math

In [10]:
term = "House of Cards"
country = "Spain"
get_number_of_tweets(term, country)

4

In [30]:
from progressbar import ProgressBar
pbar = ProgressBar()
import snscrape.modules.twitter as sntwitter
import pandas as pd

pd.options.mode.chained_assignment = None

df = pd.read_excel("input-short1.xlsx")

countries = ["India", "USA", "Germany", "Spain", "Mexico"]
for indx in pbar(df["name"].index):
    term = df["name"][indx]
    since = str(df["release_date"][indx])
    until = str(df["seven_days_later"][indx])
    for country in countries:
        df["num_"+country][indx] = get_number_of_tweets(term, country)
df.to_excel("output-short1.xlsx")

Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_composer_source=true&include_ext_alt_text=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&q=Breathe+near%3AMexico&tweet_search_mode=live&count=100&query_source=spelling_expansion_revert_click&cursor=scroll%3AthGAVUV0VFVBaAgLqhkYqhlCMWgMCxib7P9bolEjUAFQAlABEVzJkaFYCJehgHREVGQVVMVBUAFQAVARWmDBUAAA%3D%3D&pc=1&spelling_corrections=1&ext=mediaStats%252CcameraMoment: non-200 status code, retrying
Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=

In [27]:
def get_number_of_tweets(term, country):
    # tweets_list2 = []
    count = 0
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(term+" near:"+country).get_items()):
        if i>100000:
            return i
        count = i
    return count
    # Using TwitterSearchScraper to scrape data and append tweets to list
#     for i,tweet in enumerate(sntwitter.TwitterSearchScraper('"'+term+'" since:'+
#                                                             since+" until:"+until+ " near:"+country).get_items()):
#         continue
    #     if i>1500:
    #         break
    #     tweets_list2.append([tweet.date, tweet.id, tweet.content, tweet.username])
#     num = i
    # Creating a dataframe from the tweets list above
    # tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])
    # tweets_df2.to_csv("sample.csv")
#     return num

In [15]:
import time
start_time = time.time()
print(get_number_of_tweets("She", "USA"))
time.time() - start_time

1001


7.300139427185059

In [18]:
import sys
!{sys.executable} -m pip install progressbar

  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12078 sha256=cf9f6fb0435cd121de631f02772fd912d6d080d9ae422425c8314c48a020de02
  Stored in directory: c:\users\apoor\appdata\local\pip\cache\wheels\2c\67\ed\d84123843c937d7e7f5ba88a270d11036473144143355e2747
Successfully built progressbar


In [2]:
from progressbar import ProgressBar
pbar = ProgressBar()

for x in pbar(range(1000)):
    pass

100% |########################################################################|
